In [89]:
import streamlit as st
import json
import plotly.express as px
import pandas as pd
import geopandas as gpd
import geojson
import folium
import numpy as np

dataset = 'TILL6022_Emission_Dataset.csv'
df = pd.read_csv(dataset, delimiter=',', encoding='ISO-8859-1',parse_dates = ['date'])
#df.set_index('country', inplace=True)
#sf = df[df.date == '2019-01-01']
# vf = df[df.sector == 'Total']
df_filt = df.groupby(['country','sector']).resample('MS',on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
df_filt = df_filt.reset_index
df_filt.date = pd.to_datetime()



TypeError: to_datetime() missing 1 required positional argument: 'arg'

Useful commands for geopandas
#ar = gdf[gdf.ADMIN == 'Aruba'] #To find the country Aruba
#print(ar.area)                 #Print area of country Aruba
# gdf.ADMIN['Aruba']

In [37]:
with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries.geojson") as f:
    counties = geojson.load(f)
gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)
mask = gdf.area >2    # Function to remove smaller countries to speed up 
gdf_selec = gdf.loc[mask]
gdf_selec.head()

C:\Users\timtr\AppData\Local\Temp\ipykernel_13688\777596988.py:4: UserWarning:

Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




,geometry,ADMIN,ISO_A3
1,"POLYGON ((71.04980 38.40866, 71.05714 38.40903...",Afghanistan,AFG
2,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",Angola,AGO
4,"POLYGON ((19.74777 42.57890, 19.74601 42.57993...",Albania,ALB
7,"MULTIPOLYGON (((53.86305 24.23469, 53.88860 24...",United Arab Emirates,ARE
8,"MULTIPOLYGON (((-68.65412 -54.88624, -68.65413...",Argentina,ARG


In [14]:
# map = folium.Map(
#     location=[
#         gdf_selec.geometry.bounds.miny.mean(),
#         gdf_selec.geometry.bounds.minx.mean()
#     ],
#     zoom_start=1
# )

# gjson = folium.features.GeoJson(
#     gdf_selec,
# ).add_to(map)
# map

In [71]:
# fig = px.choropleth(vf, geojson=gdf_selec, color="co2",
#                     locations="ISO-A3", featureidkey="properties.district",
#                     projection="mercator"
#                    )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()


In [66]:
fig = px.choropleth(df_filt[df_filt.sector =='Domestic Aviation'], locations= 'country',
                    locationmode= "country names",
                    color='co2',
                    #hover_name="country",
                    animation_frame= 'date'
                    #color_continuous_scale="Blues_r"
                    )
fig.update_layout(
    title_text = "CO2 emmissions transport sector",
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection_type = "equirectangular"
    )
)
fig.show()

KeyError: Timestamp('2019-01-01 00:00:00')